In [1]:
import polars as pl
import json
import statistics
import re
import random
from tqdm.auto import tqdm

from openai import OpenAI

##### Load data

In [2]:
data_dir = '/Volumes/PortableSSD/CSS/data/processed/'
#data_dir = '/users/ujan/Downloads/'

In [3]:
data_df = pl.read_csv(data_dir+'posts_2015-21_ps_min_2c_politics.csv')
data_df.head()

id,created_utc,subreddit,category,super_category,author,domain,url,title,selftext,num_comments,score,gilded,upvote_ratio
str,i64,str,str,str,str,str,str,str,str,i64,i64,i64,f64
"""589v04""",1476878103,"""The_Donald""","""politics_2019""","""politics""","""gmousasi""","""i.redd.it""","""https://i.redd…","""Just a rare ba…","""""",17,1242,0,100.0
"""589ygu""",1476879588,"""Enough_Sanders…","""politics_2019""","""politics""","""12-juin-3049""","""reddit.com""","""http://www.red…","""Bernout gets e…","""""",12,28,0,100.0
"""58a7z5""",1476883248,"""EnoughTrumpSpa…","""politics_2019""","""politics""","""TheIronTARDIS""","""np.reddit.com""","""http://np.redd…","""In case you ne…","""""",38,371,0,100.0
"""58aa2h""",1476883994,"""politics""","""politics_2019""","""politics""","""Naggers123""","""breitbart.com""","""http://www.bre…","""Poll: Hillary …","""""",170,305,0,100.0
"""58bnuv""",1476899246,"""politics""","""politics_2019""","""politics""","""Metaprinter""","""gop.com""","""https://gop.co…","""Trump asking t…","""""",38,24,0,100.0


##### save positive posts to csv from the text files

In [4]:
data_dict = {'posts': [], 'exps': []}

with open(data_dir+'gpt3_pos_sample.txt') as f:
    for line in f.readlines():
        if line[0] == 'p':
            post = line.split('post : ')[-1].split('\n')[0]
            data_dict['posts'].append(post)
        elif line[0] == 'm':
            exp = line.split('meta : ')[-1].split('\n')[0]
            data_dict['exps'].append(exp)

pos_df = pl.from_dict(data_dict)
pos_df.write_csv(data_dir+'gpt3_pos_sample.csv', separator=",")

##### save negative posts to csv from the text files

In [5]:
data_dict = {'posts': []}

with open(data_dir+'gpt3_neg_sample.txt') as f:
    for line in f.readlines():
        if line[0] == 'p':
            post = line.split('post : ')[-1].split('\n')[0]
            data_dict['posts'].append(post)

neg_df = pl.from_dict(data_dict)
neg_df.write_csv(data_dir+'gpt3_neg_sample.csv', separator=",")

##### store 402 samples into df

In [20]:
samples_df = data_df.filter(pl.col('title').is_in(posts)).unique(subset=["title"]).select(["id", "title"])

samples_df.write_csv(data_dir+'chatgpt_samples.csv', separator=",")

##### chatgpt to find metaphors on the samples

In [5]:
# gpt3.5 client
client = OpenAI(api_key="api_key")  # "api_key"

#### Temperature = 0.53

##### load data and run chatgpt

In [35]:
temp = 0.53
all_responses = []

samples_df = pl.read_csv(data_dir+'chatgpt_samples.csv')

posts = samples_df['title'].to_list()
ids = samples_df['id'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Note that sports related words may be used in a nonmetaphorical way, do not label such cases as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/402 [00:00<?, ?it/s]

##### store responses and manually add gt

In [73]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts
}

responses_df = pl.from_dict(data_dict)
responses_df.write_csv(data_dir+'gpt3_responses.csv', separator=",")

##### load responses with gt

In [85]:
responses_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')
responses_df.head()

id,contains_sports_metaphor,sports_meta,explanation,ground_truth,post
str,bool,str,str,bool,str
"""6ndhz6""",false,null,null,false,"""Surprise! Wome…"
"""5zopk8""",true,"""flex its muscl…","""Assert authori…",true,"""Graham: Congre…"
"""6a8vgu""",false,null,null,false,"""Trump Removes …"
"""8vllco""",true,"""set his sights…","""Targeting or f…",true,"""Fired FBI Dire…"
"""e6nbyn""",false,null,null,false,"""Done deal: Cal…"


##### evaluate

In [ ]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()
gt = responses_df['ground_truth'].to_list()

##### total posts marked as sports metaphors by gpt

In [97]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 170


##### false positives

In [98]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 51
fp rate: 0.3


##### false negatives

In [103]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 27
fn rate: 0.11688311688311688


#### Temperature = 0.5

##### load data and run chatgpt

In [104]:
temp = 0.50
all_responses = []

samples_df = pl.read_csv(data_dir+'chatgpt_samples.csv')

posts = samples_df['title'].to_list()
ids = samples_df['id'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Note that sports related words may be used in a nonmetaphorical way, do not label such cases as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/402 [00:00<?, ?it/s]

##### save responses into df and remove one empty post

In [114]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts
}

responses_df = pl.from_dict(data_dict)
responses_df = responses_df.filter(~pl.col('post').is_in(['[deleted by user]']))

##### add gt to df

In [119]:
responses_df = responses_df.with_columns(pl.Series(name="ground_truth", values=gt)) 

##### evaluate

In [121]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphors by gpt

In [122]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 168


##### false positives

In [123]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 53
fp rate: 0.31547619047619047


##### false negatives

In [124]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 31
fn rate: 0.13304721030042918


#### Temperature = 0.45

##### load data and run chatgpt

In [126]:
temp = 0.45
all_responses = []

samples_df = pl.read_csv(data_dir+'chatgpt_samples.csv')

posts = samples_df['title'].to_list()
ids = samples_df['id'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Note that sports related words may be used in a nonmetaphorical way, do not label such cases as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/402 [00:00<?, ?it/s]

##### save responses into df and remove one empty post

In [127]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts
}

responses_df = pl.from_dict(data_dict)
responses_df = responses_df.filter(~pl.col('post').is_in(['[deleted by user]']))

##### add gt to df

In [128]:
responses_df = responses_df.with_columns(pl.Series(name="ground_truth", values=gt)) 

##### evaluate

In [129]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphors by gpt

In [130]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 170


##### false positives

In [131]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 51
fp rate: 0.3


##### false negatives

In [132]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 27
fn rate: 0.11688311688311688


#### Temperature = 0.60

##### load data and run chatgpt

In [133]:
temp = 0.60
all_responses = []

samples_df = pl.read_csv(data_dir+'chatgpt_samples.csv')

posts = samples_df['title'].to_list()
ids = samples_df['id'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Note that sports related words may be used in a nonmetaphorical way, do not label such cases as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/402 [00:00<?, ?it/s]

##### save responses into df and remove one empty post

In [134]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts
}

responses_df = pl.from_dict(data_dict)
responses_df = responses_df.filter(~pl.col('post').is_in(['[deleted by user]']))

##### add gt to df

In [135]:
responses_df = responses_df.with_columns(pl.Series(name="ground_truth", values=gt)) 

##### evaluate

In [136]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphors by gpt

In [137]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 177


##### false positives

In [138]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 56
fp rate: 0.3163841807909605


##### false negatives

In [139]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 25
fn rate: 0.11160714285714286


##### save

In [142]:
responses_df.write_csv(data_dir+'gpt3_responses_with_gt_6.csv', separator=",")

#### New prompt

In [13]:
temp = 0.53
all_responses = []

samples_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')

posts = samples_df['post'].to_list()
ids = samples_df['id'].to_list()
gt = samples_df['ground_truth'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports metaphor or not. Do not identify sports related words or phrases used in the literal sense as sports metaphors. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/401 [00:00<?, ?it/s]

##### save responses into df

In [16]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts,
    'ground_truth': gt
}

responses_df = pl.from_dict(data_dict)

##### evaluate

In [20]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphors by gpt

In [21]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 147


##### false positives

In [22]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 40
fp rate: 0.272108843537415


##### false negatives

In [23]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 39
fn rate: 0.15354330708661418


##### save

In [25]:
responses_df.write_csv(data_dir+'gpt3_responses_with_gt_new_prompt.csv', separator=",")

#### sports related prompt

In [27]:
temp = 0.53
all_responses = []

samples_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')

posts = samples_df['post'].to_list()
ids = samples_df['id'].to_list()
gt = samples_df['ground_truth'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title, and your task is to identify if the post contains a sports or sports related metaphor or not. Do not identify sports related words or phrases used in the literal sense as sports metaphors. If the text does contain a sports or sports related metaphor, identify the sports or sports related metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/401 [00:00<?, ?it/s]

##### save responses into df

In [28]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts,
    'ground_truth': gt
}

responses_df = pl.from_dict(data_dict)

##### evaluate

In [29]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphors by gpt

In [30]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 119


##### false positives

In [31]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 31
fp rate: 0.2605042016806723


##### false negatives

In [32]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 58
fn rate: 0.20567375886524822


##### save

In [33]:
responses_df.write_csv(data_dir+'gpt3_responses_with_gt_sports_rel_prompt.csv', separator=",")

In [ ]:
# You will be provided with a Reddit post title.
# Identify if the post contains a sports metaphor or not.
# Do not identify sports related words or phrases used in the literal sense as sports metaphors.
# If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation.
# Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation.

In [ ]:
# prompt 2

# You will be provided with a Reddit post title.
# Identify if the post contains a sports metaphor or not.
# If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation.
# Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation.

In [ ]:
# prompt 3

# You will be provided with a Reddit post title.
# Identify if the title is framed using sports language or not.
# If the title is framed using sports language, identify the title and provide a max 10 word explanation.
# Provide the answer in a JSON format with the following keys, contains_sports_language (true/false), explanation.

#### Prompt 2

In [6]:
temp = 0.53
all_responses = []

samples_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')

posts = samples_df['post'].to_list()
ids = samples_df['id'].to_list()
gt = samples_df['ground_truth'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title. Identify if the post contains a sports metaphor or not. If the text does contain a sports metaphor, identify the sports metaphor word or phrase and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_metaphor (true/false), sports_metaphor, explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/401 [00:00<?, ?it/s]

##### save responses into df

In [7]:
data_dict = {
    'id': ids,
    'contains_sports_metaphor': [r['contains_sports_metaphor'] for r in all_responses],
    'sports_meta': [r['sports_metaphor'] if 'sports_metaphor' in r else '' for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts,
    'ground_truth': gt
}

responses_df = pl.from_dict(data_dict)

##### evaluate

In [9]:
gpt_output = responses_df['contains_sports_metaphor'].to_list()

##### total posts marked as sports metaphor by gpt

In [10]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 155


##### false positives

In [11]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 45
fp rate: 0.2903225806451613


##### false negatives

In [12]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 36
fn rate: 0.14634146341463414


#### Prompt 3

In [13]:
temp = 0.53
all_responses = []

samples_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')

posts = samples_df['post'].to_list()
ids = samples_df['id'].to_list()
gt = samples_df['ground_truth'].to_list()

bar = tqdm(range(len(posts)))
for post in posts:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_format={ "type": "json_object" },
        seed=20,
        temperature=temp,
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a Reddit post title. Identify if the title is framed using sports language or not. If the title is framed using sports language, identify the title and provide a max 10 word explanation. Provide the answer in a JSON format with the following keys, contains_sports_language (true/false), explanation."
            },
            {
                "role": "user",
                "content": post
            }
        ],
    )
                
    obj = json.loads(response.json())
    resp_json = json.loads(obj["choices"][0]["message"]["content"])
    resp_json["post"] = post
    all_responses.append(resp_json)
    bar.update(1)

  0%|          | 0/401 [00:00<?, ?it/s]

##### save responses into df

In [14]:
data_dict = {
    'id': ids,
    'contains_sports_language': [r['contains_sports_language'] for r in all_responses],
    'explanation': [r['explanation'] if 'explanation' in r else '' for r in all_responses],
    'post': posts,
    'ground_truth': gt
}

responses_df = pl.from_dict(data_dict)

##### evaluate

In [16]:
gpt_output = responses_df['contains_sports_language'].to_list()

##### total posts marked as sports language by gpt

In [17]:
gpt_pos = 0
for val in gpt_output:
    if val:
        gpt_pos += 1

print('total posts marked as sports metaphors by gpt: {}'.format(gpt_pos))

total posts marked as sports metaphors by gpt: 21


##### false positives

In [18]:
fp = 0
for i in range(len(gpt_output)):
    if gpt_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/gpt_pos))

total fp: 10
fp rate: 0.47619047619047616


##### false negatives

In [23]:
fn = 0
for i in range(len(gpt_output)):
    if not gpt_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-gpt_pos)))

total fn: 135
fn rate: 0.35526315789473684


#### Sem matching (no FrameBERT)

In [62]:
sem_df = pl.read_csv(data_dir+'sem_matching_responses_eval.csv')
sem_df.head()

id,post,ground_truth,result,sports_meta
str,str,bool,bool,str
"""6ndhz6""","""surprise women…",false,false,""""""
"""5zopk8""","""graham congres…",true,false,""""""
"""6a8vgu""","""trump removes …",false,false,""""""
"""8vllco""","""fired fbi dire…",true,false,""""""
"""e6nbyn""","""done deal calg…",false,false,""""""


In [63]:
sem_output = sem_df['result'].to_list()
gt = sem_df['ground_truth'].to_list()

In [64]:
sem_pos = 0
for val in sem_output:
    if val:
        sem_pos += 1

print('total posts marked as sports metaphors by sem matching: {}'.format(sem_pos))

total posts marked as sports metaphors by sem matching: 47


##### false positives

In [65]:
fp = 0
for i in range(len(sem_output)):
    if sem_output[i] and not gt[i]:
        fp += 1

print('total fp: {}'.format(fp))
print('fp rate: {}'.format(fp/sem_pos))

total fp: 12
fp rate: 0.2553191489361702


##### false negatives

In [66]:
fn = 0
for i in range(len(sem_output)):
    if not sem_output[i] and gt[i]:
        fn += 1

print('total fn: {}'.format(fn))
print('fn rate: {}'.format(fn/(len(gt)-sem_pos)))

total fn: 111
fn rate: 0.3135593220338983


#### FrameBERT filter on chatgpt output

##### temp = 0.53

In [33]:
responses_df = pl.read_csv(data_dir+'gpt3_responses_with_gt_53.csv')
frame_df = pl.read_csv(data_dir+'sample_predictions_gpt_53.tsv', separator='\t')

##### get unigram metaphors

In [ ]:
NO_META_LIST = ['out', 'up', 'tip', 'check']

with open(data_dir+'meta_dict_full.json', 'r') as fp:
        data = json.load(fp)
meta_list = []
for key, values in data.items():
    meta_list.extend(values)
# remove duplicates
meta_list = list(set(meta_list))
# filter metaphors
meta_list = [meta.replace("'", '') for meta in meta_list]
meta_list = [re.sub(r"[^a-zA-Z0-9]+", ' ', meta).lower() for meta in meta_list]
meta_list = [m for m in meta_list if m not in NO_META_LIST]

uni_metas = []

for meta in meta_list:
    if len(meta.split()) == 1:
        uni_metas.append(meta)

##### get posts with unigram metas

In [46]:
uni_meta_df = responses_df.filter(pl.col('sports_meta').is_in(uni_metas))
uni_meta_df

id,contains_sports_metaphor,sports_meta,explanation,ground_truth,post
str,bool,str,str,bool,str
"""jmgevc""",true,"""underdog""","""refers to a te…",true,"""FiveThirtyEigh…"
"""3uww8c""",true,"""game""","""Refers to the …",true,"""Russia hits ba…"
"""39cs9k""",true,"""underdog""","""Referring to a…",true,"""Bernie Sanders…"
"""cm0qti""",true,"""punt""","""Avoiding addre…",true,"""Texas GOPers P…"
"""2talrj""",true,"""outfox""","""To outsmart or…",true,"""Republicans ou…"


#### FrameBERT filter on sem matching output

In [67]:
sem_df = pl.read_csv(data_dir+'sem_matching_responses_eval.csv')
frame_df = pl.read_csv(data_dir+'sample_predictions_sem_match.tsv', separator='\t')

##### get unigram metaphors

In [68]:
NO_META_LIST = ['out', 'up', 'tip', 'check']

with open(data_dir+'meta_dict_full.json', 'r') as fp:
        data = json.load(fp)
meta_list = []
for key, values in data.items():
    meta_list.extend(values)
# remove duplicates
meta_list = list(set(meta_list))
# filter metaphors
meta_list = [meta.replace("'", '') for meta in meta_list]
meta_list = [re.sub(r"[^a-zA-Z0-9]+", ' ', meta).lower() for meta in meta_list]
meta_list = [m for m in meta_list if m not in NO_META_LIST]

uni_metas = []

for meta in meta_list:
    if len(meta.split()) == 1:
        uni_metas.append(meta)

In [69]:
sem_df.head()

id,post,ground_truth,result,sports_meta
str,str,bool,bool,str
"""6ndhz6""","""surprise women…",false,false,""""""
"""5zopk8""","""graham congres…",true,false,""""""
"""6a8vgu""","""trump removes …",false,false,""""""
"""8vllco""","""fired fbi dire…",true,false,""""""
"""e6nbyn""","""done deal calg…",false,false,""""""


##### get posts with unigram metas

In [75]:
uni_meta_df = sem_df.filter(pl.col('result'))
uni_meta_df.head()

id,post,ground_truth,result,sports_meta
str,str,bool,bool,str
"""h93utk""","""economy is off…",true,true,"""off to the rac…"
"""44z4w5""","""jeb bush gets …",false,true,"""bush"""
"""b5lx5v""","""trump and repu…",true,true,"""turn the table…"
"""j06sti""","""activists to s…",true,true,"""play hardball"""
"""a962r0""","""the only probl…",true,true,"""score"""


##### preprocess framebert output

In [71]:
frame_df = frame_df.drop_nulls()

In [90]:
frame_df.head()

Tokens,Borderline_metaphor,Real_metaphors,Frame_label
str,i64,i64,str
"""surprise""",0,0,"""Experiencer_ob…"
"""womens""",0,0,"""_"""
"""marchers""",0,0,"""Change_of_lead…"
"""hired""",0,0,"""Hiring"""
"""armed""",0,0,"""Bearing_arms"""


In [100]:
tokens = frame_df['Tokens'].to_list()
metas = frame_df['Real_metaphors'].to_list()
frames = frame_df['Frame_label'].to_list()

posts = []
gen_meta = []
frame_label = []

sen = []
meta = []
frame = []
for t in range(len(tokens)):
    sen.append(tokens[t])
    if metas[t] == 1:
        meta.append(tokens[t])
        frame.append(frames[t])
    if tokens[t] == '.':
        post = " ".join(sen)
        posts.append(post)
        sen = []
        if len(meta) > 1:
            gen_meta.append(', '.join(meta))
            frame_label.append(', '.join(frame))
        elif len(meta) == 1:
            gen_meta.append(meta[0])
            frame_label.append(frame[0])
        else:
            gen_meta.append(None)
            frame_label.append(None)
        meta = []
        frame = []

new_frame_df = pl.from_dict(
    {
        'post': posts,
        'gen_meta': gen_meta,
        'frame': frame_label,
    }
)

new_frame_df.head()

post,gen_meta,frame
str,str,str
"""surprise women…",null,null
"""graham congres…","""flex, muscle, …","""Manipulation, …"
"""trump removes …","""removes""","""Removing"""
"""fired fbi dire…","""sights""","""_"""
"""done deal calg…",null,null


In [107]:
sports_meta = []

for item in gen_meta:
    if item is not None:
        metas = item.split(',')
        val = False
        for meta in metas:
            if meta in uni_metas:
                val = True
                break
        sports_meta.append(val)
                
    else:
        sports_meta.append(False)

In [108]:
len(sports_meta)

401

In [109]:
sports_meta

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
